# Nettoyage et préparation des données Amazon India pour Power BI

Produit 3 fichiers CSV :
- `amazon_clean.csv` : table principale (produits dédupliqués, colonnes dérivées)
- `amazon_reviews.csv` : reviews éclatées avec analyse de sentiment
- `amazon_words.csv` : fréquence des mots 

In [1]:
import pandas as pd
import numpy as np
import re
import string
from textblob import TextBlob
from nltk.corpus import stopwords

## 0. Chargement 

In [2]:
input = "/Users/mac/Desktop/projet_viz/amazon.csv"
out_clean = "/Users/mac/Desktop/projet_viz/outputs/amazon_clean.csv"
out_reviews = "/Users/mac/Desktop/projet_viz/outputs/amazon_reviews.csv"
out_words = "/Users/mac/Desktop/projet_viz/outputs/amazon_words.csv"

df = pd.read_csv(input)
print(f"Lignes chargées : {len(df)}")
print(f"Colonnes : {df.columns}")

Lignes chargées : 1465
Colonnes : Index(['product_id', 'product_name', 'category', 'discounted_price',
       'actual_price', 'discount_percentage', 'rating', 'rating_count',
       'about_product', 'user_id', 'user_name', 'review_id', 'review_title',
       'review_content', 'img_link', 'product_link'],
      dtype='object')


## 1.1 Nettoyage des prix (₹ et virgules)

In [3]:
# Remplacer les symboles ₹ et les virgules par des espaces vides: 
for col in ["discounted_price", "actual_price"]:
    df[col] = df[col].astype(str).str.replace("\u20b9", "", regex=False).str.replace(",", "", regex=False)
    df[col] = pd.to_numeric(df[col], errors="coerce")

## 1.2 Nettoyage du pourcentage de remise

In [4]:
# Supprimer le symbole % :
df["discount_percentage"] = (
    df["discount_percentage"].astype(str).str.replace("%", "", regex=False)
)
df["discount_percentage"] = pd.to_numeric(df["discount_percentage"], errors="coerce")

## 1.3 Nettoyage du rating

In [5]:
df["rating"].value_counts()

rating
4.1    244
4.3    230
4.2    228
4.0    129
3.9    123
4.4    123
3.8     86
4.5     75
4       52
3.7     42
3.6     35
3.5     26
4.6     17
3.3     16
3.4     10
4.7      6
3.1      4
5.0      3
3.0      3
4.8      3
3.2      2
2.8      2
2.3      1
|        1
2        1
3        1
2.6      1
2.9      1
Name: count, dtype: int64

In [6]:
# Supprimer les espaces et les barres verticales (1 seule) :
df["rating"] = df["rating"].astype(str).str.strip()
df["rating"] = df["rating"].replace("|", np.nan)
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")

## 1.4 Nettoyage du rating_count

In [7]:
df["rating_count"].isna().sum()

2

In [8]:
# Supprimer les virgules :
df["rating_count"] = df["rating_count"].astype(str).str.replace(",", "", regex=False)
# Mettre les valeurs en entiers :
# Remplacer les 2 valeurs manquantes par 0 :
df["rating_count"] = pd.to_numeric(df["rating_count"], errors="coerce").fillna(0).astype(int)

## 1.5 Extraction des categories

In [9]:
df["category"].value_counts()

category
Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables                                          233
Electronics|WearableTechnology|SmartWatches                                                                                 76
Electronics|Mobiles&Accessories|Smartphones&BasicMobiles|Smartphones                                                        68
Electronics|HomeTheater,TV&Video|Televisions|SmartTelevisions                                                               63
Electronics|Headphones,Earbuds&Accessories|Headphones|In-Ear                                                                52
                                                                                                                          ... 
Electronics|Cameras&Photography|Accessories|Batteries&Chargers|BatteryChargers                                               1
Computers&Accessories|NetworkingDevices|DataCards&Dongles                                             

In [10]:
cat_split = df["category"].str.split("|")
df["main_category"] = cat_split.str[0]
df["sub_category"] = cat_split.str[1]
df["product_type"] = cat_split.str[-1]

In [11]:
df[["main_category", "sub_category", "product_type"]].value_counts()

main_category          sub_category                    product_type            
Computers&Accessories  Accessories&Peripherals         USBCables                   233
Electronics            WearableTechnology              SmartWatches                 76
                       Mobiles&Accessories             Smartphones                  68
                       HomeTheater,TV&Video            SmartTelevisions             63
                       Headphones,Earbuds&Accessories  In-Ear                       52
                                                                                  ... 
                       HomeAudio                       SoundbarSpeakers              1
                                                       MultimediaSpeakerSystems      1
                                                       Mounts                        1
                                                       Adapters                      1
Toys&Games             Arts&Crafts                

## 1.6 Gestion des doublons (garder la 1 ère occurrence par product_id)

In [12]:
n_before = len(df)
df = df.drop_duplicates(subset="product_id", keep="first").reset_index(drop=True)
print(f"Doublons supprimés : {n_before - len(df)}")
print(f"Lignes restantes :  {len(df)}")

Doublons supprimés : 114
Lignes restantes :  1351


## 1.7 Colonnes dérivées

In [13]:
# Savings
df["savings"] = df["actual_price"] - df["discounted_price"]

# Price range
bins = [0, 500, 2000, 10000, float("inf")]
labels = ["Budget <500", "Mid-range 500-2000", "Premium 2000-10000", "Luxe >10000"]
df["price_range"] = pd.cut(df["discounted_price"], bins=bins, labels=labels, right=False)

# Rating category
def rating_cat(r):
    if pd.isna(r):
        return np.nan
    if r < 3.5:
        return "Faible"
    if r < 4.2:
        return "Moyen"
    if r < 4.5:
        return "Bon"
    return "Excellent"

df["rating_category"] = df["rating"].apply(rating_cat)

# Review count (nombre d'avis par produit)
df["review_count"] = df["review_id"].astype(str).apply(
    lambda x: len(x.split(",")) if x != "nan" else 0
)

## 1.8 Analyse de sentiment des avis

In [14]:
print("Analyse de sentiment")

# On travaille sur le df AVANT suppression des colonnes textuelles
reviews_rows = []
for _, row in df.iterrows():
    pid = row["product_id"]
    main_cat = row["main_category"]
    titles = str(row["review_title"])
    contents = str(row["review_content"])

    title_list = [t.strip() for t in titles.split(",") if t.strip() and t.strip() != "nan"]
    content_list = [c.strip() for c in contents.split(",") if c.strip() and c.strip() != "nan"]

    # Aligner les listes (prendre la plus longue)
    n = max(len(title_list), len(content_list))
    for i in range(n):
        title = title_list[i] if i < len(title_list) else ""
        content = content_list[i] if i < len(content_list) else ""
        text = f"{title} {content}".strip()
        if not text:
            continue
        polarity = TextBlob(text).sentiment.polarity
        if polarity > 0.05:
            label = "Positif"
        elif polarity < -0.05:
            label = "Negatif"
        else:
            label = "Neutre"
        reviews_rows.append({
            "product_id": pid,
            "main_category": main_cat,
            "review_title": title,
            "review_content": content,
            "sentiment_score": round(polarity, 4),
            "sentiment_label": label,
        })

df_reviews = pd.DataFrame(reviews_rows)
print(f"Avis éclatés : {len(df_reviews)} lignes")

Analyse de sentiment
Avis éclatés : 20034 lignes


In [15]:
# Agréger le sentiment par product_id et joindre a la table principale
sentiment_agg = df_reviews.groupby("product_id").agg(
    sentiment_mean=("sentiment_score", "mean"),
    pct_positif=("sentiment_label", lambda x: round((x == "Positif").mean() * 100, 1)),
    pct_negatif=("sentiment_label", lambda x: round((x == "Negatif").mean() * 100, 1)),
).reset_index()

df = df.merge(sentiment_agg, on="product_id", how="left")

# Sentiment label au niveau produit
def prod_sentiment(score):
    if pd.isna(score):
        return np.nan
    if score > 0.05:
        return "Positif"
    if score < -0.05:
        return "Negatif"
    return "Neutre"

df["sentiment_label"] = df["sentiment_mean"].apply(prod_sentiment)

## 1.9 Génération des mots 

In [16]:
print("Génération des mots")

stop_words = set(stopwords.words("english"))
# Ajouter des stopwords specifiques au domaine
stop_words.update([
    "product", "one", "good", "great", "nice", "really", "also",
    "got", "get", "use", "used", "using", "would", "much", "well",
    "even", "still", "like", "bought", "buy", "amazon", "india",
    "https", "com", "www", "jpg", "png", "webp", "images",
    "media", "image", "sx", "sy", "ql", "fmwebp", "ref", "sr",
    "qid", "dp", "html", "http"
])

def clean_text(text):
    text = str(text).lower()
    # Supprimer les URLs
    text = re.sub(r"http\S+", "", text)
    # Supprimer la ponctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Supprimer les chiffres
    text = re.sub(r"\d+", "", text)
    words = text.split()
    return [w for w in words if w not in stop_words and len(w) > 2]

word_data = []
for _, row in df.iterrows():
    main_cat = row["main_category"]
    texts = f"{row.get('review_title', '')} {row.get('review_content', '')}"
    words = clean_text(texts)
    for w in words:
        word_data.append({"word": w, "main_category": main_cat})

df_wc = pd.DataFrame(word_data)
df_wc = df_wc.groupby(["word", "main_category"]).size().reset_index(name="frequency")
df_wc = df_wc.sort_values("frequency", ascending=False).reset_index(drop=True)
print(f"Mots uniques: {df_wc['word'].nunique()}")

Génération des mots
Mots uniques: 25616


## 1.10 Suppression des colonnes non nécessaires de la table principale

In [17]:
cols_to_drop = [
    "user_id", "review_id", "review_content", "review_title",
    "user_name", "img_link", "product_link", "about_product", "category"
]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

## 1.11 Export des 3 fichiers CSV

In [18]:
df.to_csv(out_clean, index=False)
print(f"{out_clean}  ({len(df)} lignes, {len(df.columns)} colonnes)")

df_reviews.to_csv(out_reviews, index=False)
print(f"{out_reviews}  ({len(df_reviews)} lignes)")

df_wc.to_csv(out_words, index=False)
print(f"{out_words}  ({len(df_wc)} lignes)")

/Users/mac/Desktop/projet_viz/outputs/amazon_clean.csv  (1351 lignes, 18 colonnes)
/Users/mac/Desktop/projet_viz/outputs/amazon_reviews.csv  (20034 lignes)
/Users/mac/Desktop/projet_viz/outputs/amazon_words.csv  (35174 lignes)


## Vérification rapide

In [19]:
print("Vérification amazon_clean")
print(df.dtypes)
print(f"\nValeurs nulles :\n{df.isnull().sum()}")
print(f"\nApercu :\n{df.head(3).to_string()}")

Vérification amazon_clean
product_id               object
product_name             object
discounted_price        float64
actual_price            float64
discount_percentage       int64
rating                  float64
rating_count              int64
main_category            object
sub_category             object
product_type             object
savings                 float64
price_range            category
rating_category          object
review_count              int64
sentiment_mean          float64
pct_positif             float64
pct_negatif             float64
sentiment_label          object
dtype: object

Valeurs nulles :
product_id             0
product_name           0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 1
rating_count           0
main_category          0
sub_category           0
product_type           0
savings                0
price_range            0
rating_category        1
review_count           0
sentiment_mean  